In [ ]:
#call_1 = `source activate sentinel`
#call_2 = `gdalwarp -overwrite /Users/simon/Data/DEM/srtm_39_01/srtm_39_01.tif /Users/simon/Data/DEM/srtm_39_01/srtm_39_01_reproject.tif -s_srs EPSG:4326+5773 -t_srs EPSG:4979`

In [ ]:
import PyCall

In [ ]:
rasterio = PyCall.pyimport("rasterio")
dem39 = rasterio.open("/Users/simon/Data/DEM/srtm_39_01/srtm_39_01_reproject.tif")
dem38 = rasterio.open("/Users/simon/Data/DEM/srtm_38_01/srtm_38_01_transformed.tif")

In [ ]:
dem38.meta

In [ ]:
dem39.meta

In [ ]:
transform = dem38.get_transform()
rows = collect(1:dem38.height).-1;
columns = collect(1:dem38.width).-1;
lon38 = transform[1] .+ rows .* transform[2];
lat38  = transform[4] .+ columns .* transform[6];

In [ ]:
lat38.-lat39

In [ ]:
transform = dem39.get_transform()
rows = collect(1:dem39.height).-1;
columns = collect(1:dem39.width).-1;
lon39 = transform[1] .+ rows .* transform[2];
lat39  = transform[4] .+ columns .* transform[6];

In [ ]:
lon38

In [ ]:
lon39

In [ ]:
lon = cat(lon38,lon39,dims=1)
heights = hcat(dem38.read(1),dem39.read(1))

In [ ]:
import JLD

In [ ]:
JLD.save("/Users/simon/Data/DEM/denmark.jld", "lat", lat38, "lon", lon, "height", heights)

In [ ]:
import Colors

In [ ]:
test = heights;
test = convert.(Float64,test)
test[test .<0] .= 0.1 
test[test .>0.5] .= 0.9;

In [ ]:
Colors.Gray.(test[1:10:end,1:10:end])

# Huston

In [1]:
import Colors
import JLD
import PyCall
import FileIO

In [2]:
rasterio = PyCall.pyimport("rasterio")
dem76 = rasterio.open("/Users/simon/Data/DEM/srtm_17_06/srtm_17_06_reproject.tif")
dem77 = rasterio.open("/Users/simon/Data/DEM/srtm_17_07/srtm_17_07_reproject.tif")
dem86 = rasterio.open("/Users/simon/Data/DEM/srtm_18_06/srtm_18_06_reproject.tif")
dem87 = rasterio.open("/Users/simon/Data/DEM/srtm_18_07/srtm_18_07_reproject.tif")

PyObject <open DatasetReader name='/Users/simon/Data/DEM/srtm_18_07/srtm_18_07_reproject.tif' mode='r'>

In [3]:
function get_lat_lon(dem)
    transform = dem.get_transform()
    rows = collect(1:dem.height).-1;
    columns = collect(1:dem.width).-1;
    lon = transform[1] .+ rows .* transform[2];
    lat  = transform[4] .+ columns .* transform[6];
    return lat,lon
end

get_lat_lon (generic function with 1 method)

In [48]:
heights = vcat(hcat(dem76.read(1),dem86.read(1)),hcat(dem77.read(1),dem87.read(1)));

In [49]:
size(heights)

(12000, 12000)

In [47]:
test = heights[1:10:end,1:10:end];
test = convert.(Float64,test)
test[test .<0] .= 0.1 
test[test .>0.5] .= 0.9;
FileIO.save("test.jpg",Colors.Gray.(test));

In [50]:
lat_top,lon_left = get_lat_lon(dem76)
lat_bot,lon_right = get_lat_lon(dem87)
lat = cat(lat_top,lat_bot,dims=1);
lon = cat(lon_left,lon_right,dims=1);

In [51]:
JLD.save("/Users/simon/Data/DEM/houston_big.jld", "lat", lat, "lon", lon, "height", heights)

In [52]:
lat_max = 31
lat_min = 28.5
lon_max = -94
lon_min = -97;


In [53]:
idx1 = lat_min .< lat .< lat_max
idx2 = lon_min .< lon .< lon_max;

In [54]:
heights_small = heights[idx1,idx2];

In [55]:
size(heights_small)

(3000, 3599)

In [43]:
test = heights_small;
test = convert.(Float64,test)
test[heights_small .<-150] .= 0.01
test[ -140 .< heights_small .< 1] .= 0.2
test[  1 .< heights_small .< 11] .= 0.5
test[heights_small .> 10] .= 0.9;
FileIO.save("test_small.jpg",Colors.Gray.(test));

In [57]:
JLD.save("/Users/simon/Data/DEM/houston_small.jld", "lat", lat[idx1], "lon", lon[idx2], "height", heights[idx1,idx2])